In [67]:
import json
import os

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import S4

# Constants remain the same
h = 6.626070e-34  
c = 2.997925e8    
k_B = 1.380649e-23 
q = 1.602176e-19  
e_0 = 8.8541878128e-12

def Blackbody(lambda_i, T):
    return (2*h*c**2) / ((np.exp((h*c)/(k_B*T*lambda_i*1e-6))-1)*lambda_i**5)*1e14

def nb_B(lambda_i, T):
    return (2*c) / ((np.exp((h*c)/(k_B*T*lambda_i*1e-6))-1)*lambda_i**4)*1e8

def IQE(wavelength, e_g):
    # lambda_g = np.ceil(1240 / e_g) / 1000.0
    # if (lambda_g > wavelength[-1]):
    #     l_index = len(wavelength) - 1
    # else:
    #     l_index = np.where(wavelength >= lambda_g)[0][0]
    # IQE = np.ones(len(wavelength))
    # IQE[l_index:] = 0
    IQE = np.array([1, 0]) # This was created to handle the 1.705, 1.706 micron wavelengths
    return IQE

def JV(em, IQE, lambda_i, T_PV=300):
    J_L = q * np.sum(em * nb_B(lambda_i, 2073.15) * IQE) * (lambda_i[1] - lambda_i[0])
    J_0 = q * np.sum(nb_B(lambda_i, T_PV) * IQE) * (lambda_i[1] - lambda_i[0])

    V_oc = (k_B * T_PV / q) * np.log(J_L / J_0 + 1)
    t = np.linspace(0, 1, 100)
    V = t * V_oc

    J = J_L - J_0 * (np.exp(q * V / (k_B * T_PV)) - 1)
    P = V * J

    return J_L, J_0, V_oc, np.max(P)

def power_ratio(lambda_i, emissivity, T_emitter, E_g_PV=0.726, T_PV=300):
    P_emit = np.sum(emissivity * Blackbody(lambda_i, T_emitter)) * (lambda_i[1] - lambda_i[0])
    IQE_PV = IQE(lambda_i, E_g_PV)
    J_L, J_0, V_oc, P_max = JV(emissivity, IQE_PV, lambda_i, T_PV)
    FOM = J_L / P_emit
    return FOM, J_L, P_emit, J_0, V_oc, P_max

# New function to find latest iteration for a given image
def get_latest_iteration(log_dir, image_index):
    files = os.listdir(log_dir)
    iterations = [int(f.split('.')[1]) for f in files 
                 if f.startswith(f"{image_index}.") and f.endswith('.npy')]
    return max(iterations) if iterations else None

# Load from logs
base_log_dir = os.path.join('..', 'logs')

# Rest of the analysis code remains the same
wavelengths = np.linspace(0.350, 3.0, 2651)
wavelengths = wavelengths[(wavelengths != 0.5) & (wavelengths != 1.0)]
wavelengths = np.array([1.705, 1.706])
wavelengths = np.linspace(1.650, 1.750, 101)
wavelengths=np.array([.350])
T_e = 2073.15  # K
T_PV = 300     # K
E_g_PV = 0.726 # eV


# Create plots
nb_B_e = nb_B(wavelengths, T_e)
IQE_PV = IQE(wavelengths, E_g_PV)

config = {
    "num_images": int(1),
    "hidden_dimension": int(10),
    "noise_dimension": int(3),
    "seeds": {
        "torch": int(42),
        "numpy": int(42)
    },
    "default_gradient_scale": float(1e2),
    "learning_rate": float(7e-3),
    "binarization_scale": float(1e-11),
    "off_angle": float(45.0),
}

class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.noise = torch.rand(
            size=(config['num_images'], config['noise_dimension']))
        self.FC = nn.Sequential(
            nn.Linear(in_features = config['noise_dimension'],
            out_features=100),
        )

    def forward(self):
        output = self.FC(self.noise)
        output = nn.Sigmoid()(output)
        output = output * 1.1 - 0.05
        return torch.clamp(output, min=0.0, max=1.0)


generator = Generator()
generated_images = generator()

In [72]:
n_all = np.load('/home/rliacobacci/Downloads/n_allHTMats.npz')
k_all = np.load('/home/rliacobacci/Downloads/k_allHTMats.npz')

w_n = n_all['arr_0'][:, -1] + k_all['arr_0'][:, -1] * 1j
aln_n = n_all['arr_0'][:, 17] + k_all['arr_0'][:, 17] * 1j

z_step = .1
aln_depth = .473*2
z_max = 3+aln_depth
z_min = -z_step

grating_z_space = torch.linspace(1 + z_step, z_max - 2 - z_step, 4)
grating_x_space = torch.linspace(-0.5, 0.49, 100) + .5  # Every 10nm

harmonics = 14
transmitted_power_per_wavelength = np.zeros(wavelengths.shape[0])
angled_e_fields_per_wavelength = np.zeros((wavelengths.shape[0], grating_z_space.shape[0], grating_x_space.shape[0], 3), dtype=complex)

for i, wavelength in enumerate(wavelengths):
    # TODO: change to 14 when actually using the grating
    S = S4.New(Lattice=((1, 0), (0, 1)), NumBasis=harmonics)

    S.SetMaterial(Name='AluminumNitride', Epsilon=(aln_n[i]) ** 2)
    S.SetMaterial(Name='Tungsten', Epsilon=(w_n[i])**2)
    S.SetMaterial(Name='Vacuum', Epsilon=(1 + 0j)**2)

    S.AddLayer(Name='AirAbove', Thickness=1.5, Material='Vacuum')
    # permittivity_values = generated_images[0] * (aln_n[i]**2 - 1) + 1

    # width = (.5 - -.5) / len(permittivity_values)

    # n_squares = len(permittivity_values)
    # centers = torch.linspace(-.5 + width /
    #                             2, .5 - width / 2, n_squares) + .5

    # S.AddLayer(Name='Grating', Thickness=aln_depth,
    #             Material='AluminumNitride')

    # for q in range(0, 96):
    #     S.SetMaterial(Name=f'Material_{q}', Epsilon=permittivity_values[q].item())
    #     S.SetRegionRectangle(Layer='Grating', Material=f'Material_{q}', Center=(centers[q], 0), Halfwidths=(
    #         # NOTE: if this becomes -1 this works, otherwise it doesn't...??)
    #         1/200, 1 / 2), Angle=0)
    
    S.AddLayer(Name = 'AlN', Thickness = aln_depth, Material='AluminumNitride')

    S.AddLayer(Name='TungstenBelow', Thickness=1, Material='Vacuum')
    S.AddLayer(Name="AirBelow", Thickness=1, Material='Vacuum')

    S.SetExcitationPlanewave(
        IncidenceAngles=(
            # polar angle in [0,180) -- this is the first one that we change for the angular dependence
            config['off_angle'],
            0  # azimuthal angle in [0,360)
        ), sAmplitude=0, pAmplitude=1, Order=0
    )

    S.SetOptions(
        PolarizationDecomposition=True
    )

    S.SetFrequency(1 / float(wavelength))
    (norm_forw, norm_back) = S.GetPowerFluxByOrder(Layer='TungstenBelow', zOffset=0)[0]

    transmitted_power_per_wavelength[i] = np.abs(norm_forw)

    print(f'{np.round(wavelength * 1000)}nm: {transmitted_power_per_wavelength[i]}')
    
    print(S.GetPowerFluxByOrder(Layer = 'AirAbove',  zOffset=0)[1])

    zc = 0
    for z in grating_z_space:
        # TODO: verify that the order of the responses matches the natural order of the x variables
        E, H = S.GetFieldsOnGrid(
            z, NumSamples=(200, 1), Format='Array')
        angled_e_fields_per_wavelength[i][zc] = np.array(E[0])[1::2]
        zc += 1

    del S
    x = len(np.where(np.real(np.mean(np.einsum('ijkl,ijkl->ijk', angled_e_fields_per_wavelength, angled_e_fields_per_wavelength)[i], axis = 0)) > 0)[0])
    y = np.real(np.mean(np.einsum('ijkl,ijkl->ijk', angled_e_fields_per_wavelength, angled_e_fields_per_wavelength)[i], axis = 0))
    if x != 100 and x != 0:
        print("We found one!")

350.0nm: 0.5630732646969361
(0j, 0j)


In [60]:
print(np.mean(np.einsum('ijkl,ijkl->ijk', angled_e_fields_per_wavelength, angled_e_fields_per_wavelength)[0], axis=0))
print(angled_e_fields_per_wavelength.shape)

(4, 100)
(101, 4, 100, 3)


In [6]:
def plot_contour(image_array, title = '', output_path = '', show_plot=False):
    """
    Plot a single image array as a grid of colored squares and save it.
    """
    plt.figure(figsize=(8, 6))
    ax = plt.gca()

    # Check if image_array is 2D
    if image_array.ndim == 1:
        print(f"Image data is 1D with shape {image_array.shape}. Attempting to reshape to 2D.")
        # Attempt to reshape to 2D if possible
        sqrt_len = int(np.sqrt(len(image_array)))
        if sqrt_len * sqrt_len == len(image_array):
            image_array = image_array.reshape((sqrt_len, sqrt_len))
            print(f"Successfully reshaped to 2D with shape {image_array.shape}.")
        else:
            print(f"Cannot reshape image data of length {len(image_array)} to a square 2D array.")
            print("Skipping this image. Ensure your images are 2D or have dimensions that can be reshaped appropriately.")
            plt.close()
            return
    elif image_array.ndim != 2:
        print(f"Unsupported image array with {image_array.ndim} dimensions. Only 2D arrays are supported.")
        plt.close()
        return

    # Define the colormap from white (0) to black (1)
    cmap = plt.cm.gray_r  # 'gray_r' reverses the 'gray' colormap

    # Create the grid of squares with black edges
    try:
        mesh = ax.pcolormesh(image_array, cmap=cmap, edgecolors='black', linewidth=0.1, shading='auto', vmin=0, vmax=1)
    except Exception as e:
        print(f"Failed to create grid plot for {title}: {e}")
        plt.close()
        return

    plt.title(title)
    plt.xlabel('X-axis')
    plt.ylabel('Y-axis')
    ax.set_aspect('equal')  # Ensure squares are square

    # Remove axis ticks for cleaner look
    ax.set_xticks([])
    ax.set_yticks([])

    # Improve layout
    plt.tight_layout()

    # Save the plot
    # try:
    #     plt.savefig(output_path, dpi=300)
    #     print(f"Saved grid plot to {output_path}")
    # except Exception as e:
    #     print(f"Failed to save plot {output_path}: {e}")

    if show_plot:
        plt.show()
    else:
        plt.close()